## ============================T-Shirt Classifier==============================

#### Importing dependencies

In [ ]:
import numpy as np
import pandas as pd
import os
import random
import PIL
import csv
import cv2
import matplotlib.pyplot as plt
import itertools
from PIL import Image
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.models import load_model
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder  
from keras.optimizers import SGD
from keras.applications.inception_v3 import InceptionV3, decode_predictions
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

#### Walking the directory to get all image files

In [ ]:
train_files = []
val_files = []
test_files = []

train_files = os.listdir('train_images')
val_files = os.listdir('val_images')
test_files = os.listdir('test_images') 

#### Displaying 6 random images

In [ ]:
Temp = np.random.choice(train_files, 6)
for i in Temp:
    Label = i.split('_')[0]    
    im = plt.imread('train_images/'+i)
    plt.imshow(im)
    plt.title(Label)
    plt.show()

#### Preprocessing the images

In [ ]:
size = 256
def PreprocessingImages(files,directory):
    x = []
    print('Preprocessing '+directory+' images ...')
    for file in files:
        Imgpath = directory+'_images/'+file
        im = Image.open(Imgpath)
        im = im.resize((size, size), PIL.Image.NEAREST)
        im = np.asarray(im, dtype='float64')
        x.append(im)
    print('Total number of '+directory+' images:', len(files))
    return(x)

In [ ]:
x_train = PreprocessingImages(train_files, 'train')
x_val = PreprocessingImages(val_files, 'val')
x_test = PreprocessingImages(test_files, 'test')
print("Done reading all images")

In [ ]:
x_train = np.array(x_train)
print(x_train.shape, 'Train Data Shape')

x_val = np.array(x_val)
print(x_val.shape, 'Val Data Shape')


x_test = np.array(x_test)
print(x_test.shape, 'Test Data Shape')

In [ ]:
#Saving the into .npy files to load quickly from next time
np.save("train_preprocessed.npy",x_train)
np.save("val_preprocessed.npy",x_val)
np.save("test_preprocessed.npy",x_test)

#### Normalizing the data

In [ ]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.
x_val /= 255.
x_test /= 255.

In [ ]:
#Saving the into .npy files to load quickly from next time
np.save("train_normalized.npy",x_train)
np.save("val_normalized.npy",x_val)
np.save('test_normalized.npy',x_test)

In [ ]:
#### Loading the saved files directly
x_train = np.load('train_normalized.npy')
x_val = np.load('val_normalized.npy')
x_test = np.load('test_normalized.npy')

#### Getting the labels from name of each images

In [ ]:
def GetLabel(files):
    y = []
    for file in files:
        y.append(file.split('_')[0])
    return(y)

In [ ]:
y_train = GetLabel(train_files)
y_val = GetLabel(val_files)
y_test = GetLabel(test_files)

In [ ]:
#Encoding the labels(from text to numbers)
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
print(dictionary)

In [ ]:
#converting labels to categorical data
y_train = to_categorical(y_train)
print(y_train.shape, 'Train data Labels Shape')

y_val = to_categorical(y_val)
print(y_val.shape, 'Val data Labels Shape')

y_test = to_categorical(y_test)
print(y_test.shape, 'Test data Labels Shape')

#### Building model using Transfer Learning(Inception V3)

In [ ]:
trained_model = InceptionV3(include_top=False,weights='imagenet')

x = trained_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128,activation='relu')(x)
x = BatchNormalization()(x)
pred_inception= Dense(2,activation='sigmoid')(x)
model = Model(inputs=trained_model.input,outputs=pred_inception)

In [ ]:
#making the layers of inception non-trainable
for layer in trained_model.layers:
    layer.trainable=False
    
#compiling the model
adam = Adam(lr=0.001)
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=adam)

In [ ]:
model.fit(x_train, y_train, batch_size= 32, epochs= 25, validation_data=(x_val, y_val))

#### Evaluating the model on test data

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save('modelV1.h5')

## Building a custom model

In [ ]:
model = Sequential()
model.add(Convolution2D(filters=32, kernel_size=(3, 3), padding='same', 
                        input_shape=(256,256,3), 
                        activation='relu'))
model.add(Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Convolution2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data=(x_val, y_val),
          shuffle=True)

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Hyper parameter tuning

In [ ]:
batch_size = 32

def model_train(optimizer, lr_rate, freez, nb_epoch=2):
    tag = 'model_'+optimizer+'_lr_'+str(lr_rate)+'_freeze_'+str(freez)
    trained_model = InceptionV3(include_top=False,weights='imagenet')
    x = trained_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    pred_inception= Dense(2,activation='sigmoid')(x)
    model = Model(inputs=trained_model.input,outputs=pred_inception)

    #making the layers non-trainable
    for layer in trained_model.layers:
        layer.trainable=freez

    #compiling the model
    if optimizer=='adam':
        optim = Adam(lr=lr_rate)
    elif optimizer=='sgd':
        optim = SGD(lr=lr_rate)
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=optim)

    train_datagen = ImageDataGenerator(rescale=None, 
                                       height_shift_range=0.2, 
                                       width_shift_range=0.2,
                                       horizontal_flip=True, 
                                       rotation_range=10,shear_range=0.2,zoom_range=0.2,)
    validation_datagen = ImageDataGenerator(rescale=None)

    train_generator = train_datagen.flow(x=x_train, y=y_train,batch_size=batch_size)
    validation_generator = validation_datagen.flow(x=x_val, y=y_val,batch_size=batch_size)

    hist = model.fit_generator(train_generator,
                        steps_per_epoch=train_generator.n//batch_size, # '//' in python returns only the quotient
                        epochs=nb_epoch,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n//batch_size, 
                        verbose=0).history
    
    model.save(tag+'.h5')
    np.save(tag+'.npy', hist)
    
    t_acc = np.min(hist['acc'])
    v_acc = np.min(hist['val_acc'])
    return(t_acc, v_acc)

In [ ]:
optimizer_list = ['adam', 'sgd']
lr_rate_list = [0.01,0.001]
freez_list = [True, False]

gs_list = list(itertools.product(optimizer_list, lr_rate_list, freez_list))
print(gs_list)

In [ ]:
tr_acc = []
val_acc = []
for op, lr, fr in gs_list:
    print('Running combination - Optimizer:{}, Learning Rate:{}, Freeze Weights:{}'.format(op, lr, fr))
    r1, r2 = model_train(op, lr, fr)
    tr_acc.append(r1)
    val_acc.append(r2)

In [ ]:
model = load_model('model_adam_lr_0.01_freeze_True.h5')

In [ ]:
df = pd.DataFrame(gs_list)
df.columns = ['Optimizer', 'Learning Rate', 'Freeze Weights']
df['Train Acc'] = tr_acc
df['Val Acc'] = val_acc

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))